In [1]:
raw_prompt = """
For each item that the buyer agreed to purchase, return the item name, item description, quantity purchased, unit price in dollars, unit discount in dollars, and final total price per item (quantity * (unit price - unit discount)) . Each sku of the purchased item should be treated as a separate item.

Output the result in json format following the example below. Each line item should have its own list under lineItems. Include all non quantity and price related details of each item under its description.

{ "orderDetails":{ "orderDate": "", "orderNum": null, }, "buyer":{ "companyName": "", "pointOfContact": "", "contactEmail": "", "contactPhone": "", "shippingAddress": "", }, "lineItems":[{ "itemNumber": null, "description": "", "quantity": null, "unitPrice": null, "discount": null, “totalPrice”: null}]}
"""

In [2]:
from midas.agent import Midas

In [3]:
new_agent = Midas()

In [4]:
new_agent.set_mission(raw_prompt)

In [5]:
new_agent.save('/Users/kaihayden/Desktop/midas/first_subquery_save.json')

In [6]:
new_agent.load('/Users/kaihayden/Desktop/midas/first_subquery_save.json')